In [11]:
import pickle
import pandas as pd

ratings = pickle.load( open( "saveKidsF.p", "rb" ) )


In [12]:
ratingsPD = pd.DataFrame(ratings, columns=['user', 'item', 'rating'] )
ratingsPD.head()

,user,item,rating
0,0,2,11.0
1,0,3,6.0
2,0,6,1.0
3,0,12,93.0
4,0,15,1.0


In [13]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = preprocessing.MinMaxScaler([1,5])
def fx(x):
    data = preprocessing.scale(x['rating'].to_numpy().reshape(-1, 1))
    scaler.fit(data)
    data=scaler.transform(data)
    x['rating']=data
    return x
ratingsPD = ratingsPD.groupby('user').apply(fx)

In [14]:
ratingsPD1= ratingsPD.groupby('user').filter(lambda x : len(x)>24)


In [15]:
ratingsPD2 = ratingsPD1.groupby('item').filter(lambda x : len(x)>3)

In [16]:
ratings =ratingsPD2

In [17]:
len(ratings)

756791

In [18]:
ratings.groupby('user').count()

,item,rating
user,,
0,1239,1239
1,294,294
2,397,397
3,938,938
4,919,919
5,414,414
6,563,563
7,1318,1318
8,232,232


In [19]:
ratings['item'].nunique()

38173

In [20]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias
rm = 0
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5,xf.SampleFrac(0.3)):
    pred = als.BiasedMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    #print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))


0.40076940568123987
0.41352212895163637
0.3916901996781433
0.4107698221054306
0.4220838754218299
Average RMSE: 0.407767086367656


In [ ]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias
rm = 0
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5,xf.SampleFrac(0.3)):
    pred = als.ImplicitMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    #print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))